In [37]:
# Generelle moduler og funksjonsbeskrivelser brukt i forelesningen
from numpy import sin, cos, pi, exp
import numpy as np
import scipy.signal as sig
import matplotlib.pyplot as plt
from Kildekode._11_Frekvensrespons import *

def displayFrequencyResponse(b, a=[1], fig_num=1):
    w, Hw = sig.freqz(b, a)
    H_amp = np.abs(Hw)
    H_phase = np.unwrap(np.angle(Hw))
    plt.close(fig_num); plt.figure(fig_num)
    plt.subplot(2,1,1)
    plt.plot(w, H_amp)
    plt.grid(True)
    plt.xticks(np.linspace(0, 1, 6)*pi, [str(round(i,2))+r'$\pi$' for i in np.linspace(0, 1, 6)])
    plt.xlabel(r'Digital Frekvens $\hat{\omega}$')
    plt.ylabel(r'$\left| H\left(e^{j\hat{\omega}}\right)\right|$')
    plt.xlim([0, pi])
    plt.title(r'Frekvensrespons $H\left(e^{j\hat{\omega}}\right)$')
    
    plt.subplot(2,1,2)
    plt.plot(w, H_phase)
    plt.grid(True)
    plt.xticks(np.linspace(0, 1, 6)*pi, [str(round(i,2))+r'$\pi$' for i in np.linspace(0, 1, 6)])
    plt.yticks(np.linspace(0, -1, 6)*pi, [str(round(i,2))+r'$\pi$' for i in np.linspace(0, -1, 6)])
    plt.xlabel(r'Digital Frekvens $\hat{\omega}$')
    plt.ylabel(r'$\angle H\left(e^{j\hat{\omega}}\right)$')
    plt.xlim([0, pi])
    plt.tight_layout(pad=0.1, w_pad=1.0, h_pad=1.0)

%matplotlib ipympl

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<img src="NTNU_Logo.png" align="left" style="width: 30%">
<br clear="all" />
<br></br>

# Frekvensrespons for FIR filtre

* **Emne IELEA2302 - Signalbehandling**
* **Uke 12, 2021**
* **Relevant Pensum:**
    * Kapittel 7 i læreboka DSP First
* **Underviser: Kai Erik Hoff**

# Tema tirsdag 23. mars

* Oppsummering frekvensrespons
* Utregning av frekvensrespons med DTFT
* DTFT egenskaper og transformasjonspar
* Nøkkelbegrep i filterdesign
* Idéelle filtre
* Vindusmetoden for tilnærming til ideelle lavpassfiltre

# Grafisk fremstilling av frekvensrespons

* Frekvensresponsen $H\left(e^{j\hat{\omega}}\right)$ er en *kompleks funksjon* som forteller hvordan filteret vil påvirke enhver ren sinussekvens $x[n] = A\cdot \cos(\hat{\omega}_0\cdot n + \phi)$.
<img src="Figurer/11_Frekvensrespons/Fig3_LTI_SInusoid.png" style="width: 80%; margin-left: 100px" />

* Når vi analyserer filtre er det gunstig å kunne studere skaleringen og forskyvningen hver for seg. Disse to "komponentene" er kjent som amplituderesponsen og faseresponsen til et filter.
    * **Amplituderespons** $\left|H\left(e^{j\hat{\omega}}\right)\right|$
        * Forteller hvor mye en sinussekvens forsterkes/dempes, gitt digital frekvens $\hat{\omega}$.
    * **Faserespons** $\angle H\left(e^{j\hat{\omega}}\right)$
        * Forteller hvor mange radianer sinussekvensen ut av filteret er forskjøvet i forhold til inngangssignalet, gitt digital frekvens $\hat{\omega}$.

# Amplituderespons

* Angir filterets skalering av sinussekvenser med digital frekvens $\hat{\omega}$.

$$\left|H\left(e^{j\hat{\omega}}\right)\right| = \frac{\left|Y\left(e^{j\hat{\omega}}\right)\right|}{\left|X\left(e^{j\hat{\omega}}\right)\right|}$$

* Lik symmetri rundt $\hat{\omega}=0$.
$$\left|H\left(e^{j\hat{\omega}}\right)\right| = \left|H\left(e^{-j\hat{\omega}}\right)\right|$$

<img src="Figurer/11_Frekvensrespons/Fig4_AmpResp.png" style="width: 80%; margin-left: 100px" />

# Faserespons
* Angir forsinkelsen av bølgeformen til en sinussekvens med digital frekvens $\hat{\omega}$ påført av filteret.
$$\angle H\left(e^{j\hat{\omega}}\right) = \angle Y\left(e^{j\hat{\omega}}\right) - \angle X\left(e^{j\hat{\omega}}\right)$$
* Oppgitt i radianer. En faserespons på $\angle H\left(e^{j\hat{\omega}}\right) =  -2\pi$ vil f.eks. tilsvare en forsinkelse på én signalperiode for signalkomponenten med frekvens $\hat{\omega}$.
* Odde symmetri rundt $\hat{\omega}=0$:
$$\angle H\left(e^{j\hat{\omega}}\right) = -\angle H\left(e^{-j\hat{\omega}}\right)$$

<img src="Figurer/11_Frekvensrespons/Fig5_PhaseResp.png" style="width: 80%; margin-left: 100px" />

## Demo: Frekvensrespons

In [20]:
hn = np.array([-0.25, 0.5, -0.25])
FreqRespDemo(hn, figsize=(12,8))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Output()

# Utregning av frekvensrespons

* Diskrét-Tids Fouriertransformasjon (DTFT) av impulsresponesn $h[n]$ gir et *kontinuerlig* funksjonsuttrykk for filterets frekvensrespons.

* Formel for *kausale* filtre:

### \begin{align}H\left(e^{j\hat{\omega}}\right) &= \sum_{n=-\infty}^{\infty} h[n]\cdot e^{-j\hat{\omega}\cdot n}\\ &= h[0]\cdot e^{0}+h[1]\cdot e^{-j\hat{\omega}}+h[2]\cdot e^{-j2\hat{\omega}}+\ldots \end{align}

## Regneeksempel 1
* Et FIR filter er gitt ved impulsresponsen $h[n] =-\frac{1}{4}\delta[n]+ \frac{1}{2}\delta[n-1]- \frac{1}{4}\delta[n-2]$
* Finn et funksjonsuttrykk for filterets frekvensrespons $H\left(e^{j\hat{\omega}}\right)$.

## Demo: frekvensrespons

In [ ]:
hn = np.array([0.25, 0.5, 0.25])
DTFT_demo(hn, fig_num = 6, figsize=(12,8))

# DTFT relevante egenskaper

|Egenskap|Tidsdomene $x[n]$|Frekvensdomene $X\left(e^{j\hat{\omega}}\right)$|
|:--|:----------:|:-----------------------:|
|Perdiodisistet i frekvensdomenet| | \begin{equation}X\left(e^{j(\hat{\omega}+2\pi)}\right)=X\left(e^{j\hat{\omega}}\right)\end{equation} |
|Linearitet | \begin{equation}a\cdot x_1[n] + b\cdot x_2[n] \end{equation}|\begin{equation}a\cdot X_1\left(e^{j\hat{\omega}}\right) +b\cdot X_2\left(e^{j\hat{\omega}}\right)\end{equation}|
|Komplekskonjugert symmetri| $x[n]$ er et reelt signal. | \begin{equation}X\left(e^{-j\hat{\omega}}\right) = X^{*}\left(e^{j\hat{\omega}}\right)\end{equation}|
|Tidsreversering | $x[-n]$ | \begin{equation}X\left(e^{-j\hat{\omega}}\right)\end{equation}|
|Tidsforskyvning | $x[n-n_d]$ | \begin{equation}e^{-j\hat{\omega}\cdot n_d} \cdot X\left(e^{-j\hat{\omega}}\right)\end{equation}|
|Frekvensforskyvning | $x[n]\cdot e^{j\hat{\omega}_0\cdot n}$ | \begin{equation}X\left(e^{j(\hat{\omega}-\hat{\omega}_0)}\right)\end{equation} | 
|Modulasjon | $x[n]\cdot \cos (\hat{\omega}_0 \cdot n)$ | \begin{equation}\frac{1}{2}X\left(e^{j(\hat{\omega}-\hat{\omega}_0)}\right) + \frac{1}{2}X\left(e^{j(\hat{\omega}+\hat{\omega}_0)}\right)\end{equation}| 
|Konvolusjon| $x[n]*h[n]$ | \begin{equation}X\left(e^{j\hat{\omega}}\right)\cdot H\left(e^{j\hat{\omega}}\right)\end{equation}|
|Multiplikasjon | $x[n]\cdot w[n]$ | \begin{equation}X\left(e^{j\hat{\omega}}\right)*W\left(e^{j\hat{\omega}}\right)\end{equation}|

# DTFT transformasjonspar

|Tidsdomene: $x[n]$ | Frekvensdomene: $X\left(e^{j\hat{\omega}}\right)$ |
|:-:|:-:|
|$\delta[n]$|$1$|
|$\delta[n-n_d]$ | $e^{-j\hat{\omega}\cdot n_d}$ |
| $$u[n+M]-u[n-M]$$ | $$ \frac{\sin\left(\frac{1}{2}(2M+1)\hat{\omega}\right)}{\sin\left(\frac{1}{2}\hat{\omega}\right)} $$|
|$$\frac{\sin(\hat{\omega}_b\cdot n}{\pi \cdot n}$$|$$\begin{cases}1, |\hat{\omega}|\leq \hat{\omega}_b\\ 0, \hat{\omega}_b < |\hat{\omega}| \leq \pi \end{cases}$$|
|$$a^n\cdot u[n], \ \ (|a|<1) $$| $$\frac{1}{1-a\cdot e^{-j\hat{\omega}}}$$|

# Praktisk filterdesign
* Design av lavpass, høypass, båndpass og båndstoppfiltre.

#### Premiss:
* Vi ønsker å finne filterkoeffisisenter som på best mulig måte oppfyller ønskede filteregenskaper, samtidig som vi unngår unødvendig høy filterorden.
    * Høyere filterorden betyr flere regneoperasjoner per sample, og høyere forsinkelse gjennom filteret.

# Sentrale begrep
* **Stoppbåndsdemping:** minimum antall dB demping av signalkomponenter i stoppbåndet.
* **Stoppbånd:** frekvensområdet der demping signalkomponenter er *minst* like stor som stoppbåndsdempingen. 
* **Passbånd rippel:** maksimal akseptabel variasjon i passbåndet.
* **Passbånd:** frekvensområdet der amplitudeendring av signalkomponenter ikke overskrider passbånd rippel.
* **Transisjonsbånd:** frekvensområdet mellom stoppbånd og passbånd.

## Illustrasjon
<img src="Figurer/12_DTFT%20og%20filterdesign/Fig1_FilterParams.png" style="width: 80%; margin-left: 100px" />

# Alt starter med lavpassfilter

* Ved å utnytte egenskapene til *amplitudemodulasjon* og *sammensatte LTI-system* kan man lage en valgfri filtertype ved å starte med et lavpassfilter.
    * Båndpassfilter og høypassfilter er amplitudemodulerte lavpassfilter.
    * Båndstoppfilter er lavpassfilter og høypassfilter i parallell.

# Ideelle lavpassfilter
* Et ideelt filter slipper gjennom ønskede frekvenskomponenter uten noen endring i amplitude eller fase, og fjerner alle tegn til de uønskede frekvenskomponentene.
* Illustrasjon ideelt filter med kantfrekvens $\hat{\omega}_c = 0.4\pi$:

<img src="Figurer/12_DTFT%20og%20filterdesign/Fig2_Ideal.png" style="width: 80%; margin-left: 100px" />

<img src="Figurer/12_DTFT%20og%20filterdesign/Slide1.png" style="width: 90%; margin-left: 50px" />

# Ny trigonometrisk funksjon: $\text{sinc}(x)$

* Definisjon:
$$\text{sinc}(x) = \frac{\sin(\pi\cdot x)}{\pi\cdot x}$$
    * Er definert for grenseverdien $x\rightarrow 0$:
    $$\lim_{x\rightarrow 0} \frac{\sin(\pi\cdot x)}{\pi\cdot x} = \frac{0}{0} = 1$$
    

* I python er denne innebygd i `numpy`-modulen som [`numpy.sinc()`](https://numpy.org/doc/stable/reference/generated/numpy.sinc.html).

<img src="Figurer/12_DTFT%20og%20filterdesign/Slide2.png" style="width: 90%; margin-left: 50px" />

<img src="Figurer/12_DTFT%20og%20filterdesign/Slide3.png" style="width: 90%; margin-left: 50px" />

## Kodeeksempel 1: 
* Finn impulsresponsen til et lavpassfilter med filterorden $M = 36$ og digital "kantfrekvens" $\hat{\omega}_c$ i $0.3\pi$.
* NB! 
$$\frac{\sin(\hat{\omega}_0\cdot n)}{\pi \cdot n} = \frac{\hat{\omega}_0}{\pi}\cdot \text{sinc}\left(\frac{\hat{\omega}_0}{\pi}\cdot n \right)$$

<img src="Figurer/12_DTFT%20og%20filterdesign/Slide4.png" style="width: 90%; margin-left: 50px" />

<img src="Figurer/12_DTFT%20og%20filterdesign/Slide5.png" style="width: 90%; margin-left: 50px" />

<img src="Figurer/12_DTFT%20og%20filterdesign/Slide6.png" style="width: 90%; margin-left: 50px" />

<img src="Figurer/12_DTFT%20og%20filterdesign/Slide7.png" style="width: 90%; margin-left: 50px" />

<img src="Figurer/12_DTFT%20og%20filterdesign/Slide8.png" style="width: 90%; margin-left: 50px" />

<img src="Figurer/12_DTFT%20og%20filterdesign/Slide9.png" style="width: 90%; margin-left: 50px" />

<img src="Figurer/12_DTFT%20og%20filterdesign/Slide10.png" style="width: 90%; margin-left: 50px" />

## Kodeeksempel 2:
* Bruk en vindusfunksjon til å øke stoppbåndsdempingen i filteret fra eksempel 1.

<img src="Figurer/12_DTFT%20og%20filterdesign/Slide11.png" style="width: 90%; margin-left: 50px" />

<img src="Figurer/12_DTFT%20og%20filterdesign/Slide12.png" style="width: 90%; margin-left: 50px" />

# Kodeeksempel 3:
* Lag et båndpassfilter med "passbånd" mellom $\hat{\omega}_{LP} = 0.3\pi$ og $\hat{\omega}_{HP} = 0.5\pi$ og filterorden 128.

# Andre spørsmål?